In [47]:
import pandas as pd
import os
print('Working Dictionary:', os.getcwd())
import ollama
import requests

Working Dictionary: E:\MLPROJECT


In [48]:
df = pd.read_parquet('cleaned_hospital_readmission.parquet', engine='pyarrow')
df

,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type,admission_sources,discharge_dispositions,primary_diagnosis_1,primary_diagnosis_2,primary_diagnosis_3
1,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,...,No,Ch,Yes,>30,Urgent/Emergency,Emergency,Home Discharge,Other,Diabetes,Other
2,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,...,No,No,Yes,NO,Urgent/Emergency,Emergency,Home Discharge,Other,Diabetes,Other
3,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,...,No,Ch,Yes,NO,Urgent/Emergency,Emergency,Home Discharge,Other,Diabetes,Circulatory
4,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,...,No,Ch,Yes,NO,Urgent/Emergency,Emergency,Home Discharge,Neoplasms,Neoplasms,Diabetes
5,82637451,Caucasian,Male,[50-60),2,1,2,3,31,6,...,No,No,Yes,>30,Urgent/Emergency,Referral-Based,Home Discharge,Circulatory,Circulatory,Diabetes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,100162476,AfricanAmerican,Male,[70-80),1,3,7,3,51,0,...,No,Ch,Yes,>30,Urgent/Emergency,Emergency,Transferred,Diabetes,Other,Circulatory
101762,74694222,AfricanAmerican,Female,[80-90),1,4,5,5,33,3,...,No,No,Yes,NO,Urgent/Emergency,Transferred,Transferred,Digestive,Other,Digestive
101763,41088789,Caucasian,Male,[70-80),1,1,7,1,53,0,...,No,Ch,Yes,NO,Urgent/Emergency,Emergency,Home Discharge,Other,Genitourinary,Other
101764,31693671,Caucasian,Female,[80-90),2,3,7,10,45,2,...,No,Ch,Yes,NO,Urgent/Emergency,Emergency,Transferred,Injury,Other,Injury


In [52]:
def call_ollama(user_prompt, model_id="llama3", system_prompt=None, options=None):
    messages = [{'role': 'user', 'content': user_prompt}]
    if system_prompt is not None:
        messages.append({'role': 'system', 'content': system_prompt})

    response = ollama.chat(
        model=model_id,
        messages=messages,
        options=options
    )
    return response['message']['content']


In [53]:
import ollama
print(ollama.list())

models=[Model(model='llama3:latest', modified_at=datetime.datetime(2025, 3, 8, 15, 53, 45, 943100, tzinfo=TzInfo(-06:00)), digest='365c0bd3c000a25d28ddbf732fe1c6add414de7275464c4e4d1c3b5fcb5d8ad1', size=4661224676, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0'))]


In [54]:
def patient_data_to_text(patient_row):
    """
    Convert structured patient data into a natural language medical summary.
    """
    return f"""
    Patient ID: {patient_row['patient_nbr']}, a {patient_row['age']} {patient_row['gender']} patient, 
    was admitted to the hospital through {patient_row['admission_sources']} under {patient_row['admission_type']}.

    - Stayed for {patient_row['time_in_hospital']} days.
    - Had {patient_row['num_lab_procedures']} lab procedures, {patient_row['num_procedures']} other procedures, 
      and was prescribed {patient_row['num_medications']} medications.
    - Past visits: {patient_row['number_outpatient']} outpatient, {patient_row['number_emergency']} emergency, 
      {patient_row['number_inpatient']} inpatient.
    - Diagnosed primarily with {patient_row['diag_1']}, secondary diagnoses include {patient_row['diag_2']} and {patient_row['diag_3']}.
    - Number of diagnoses made: {patient_row['number_diagnoses']}.
    
    **Medications & Diabetes Management:**
    - Diabetes medication: {patient_row['diabetesMed']} (Change in medication: {patient_row['change']}).
    - Insulin usage: {patient_row['insulin']}.
    - Other prescribed medications include: Metformin: {patient_row['metformin']}, Glipizide: {patient_row['glipizide']}, 
      Glyburide: {patient_row['glyburide']}, Pioglitazone: {patient_row['pioglitazone']}, Rosiglitazone: {patient_row['rosiglitazone']}, 
      Acarbose: {patient_row['acarbose']}, Miglitol: {patient_row['miglitol']}, among others.

    **Discharge Details:**
    - Discharged under {patient_row['discharge_dispositions']}.
    """

patient_summary = patient_data_to_text(df.iloc[0])
print(patient_summary)



    Patient ID: 55629189, a [10-20) Female patient, 
    was admitted to the hospital through Emergency under Urgent/Emergency.

    - Stayed for 3 days.
    - Had 59 lab procedures, 0 other procedures, 
      and was prescribed 18 medications.
    - Past visits: 0 outpatient, 0 emergency, 
      0 inpatient.
    - Diagnosed primarily with 276, secondary diagnoses include 250.01 and 255.
    - Number of diagnoses made: 9.
    
    **Medications & Diabetes Management:**
    - Diabetes medication: Yes (Change in medication: Ch).
    - Insulin usage: Up.
    - Other prescribed medications include: Metformin: No, Glipizide: No, 
      Glyburide: No, Pioglitazone: No, Rosiglitazone: No, 
      Acarbose: No, Miglitol: No, among others.

    **Discharge Details:**
    - Discharged under Home Discharge.
    


In [55]:
df["patient_summary"] = df.apply(patient_data_to_text, axis=1)


In [56]:
def classify_readmission_with_ollama(patient_summary):
    prompt = f"""
    Based on the following patient data, predict the likelihood of hospital readmission within 30 days.
    
    {patient_summary}
    
    Answer only with '1' (High) or '0' (Low).
    """
    response = call_ollama(prompt)
    return int(response.strip()) if response.strip().isdigit() else 0  # Default to '0' if invalid response


In [129]:
def classify_readmission_with_ollama(patient_summary):
    prompt = f"""
    You are a medical AI trained to predict hospital readmissions. 
    Analyze the following patient data and assess the likelihood of readmission within 30 days.

    - Consider past medical history, medications, and number of hospital visits.
    - Prioritize factors such as age, comorbidities, and past admissions.
    - Give a **binary response**: '1' for High risk of readmission and '0' for Low risk.
    
    Patient Data:
    {patient_summary}

    Respond ONLY with '1' or '0' and nothing else.
    """
    
    response = call_ollama(prompt)
    return int(response.strip()) if response.strip().isdigit() else 0  # Default to '0' if invalid response


In [116]:
def classify_readmission_with_ollama(patient_summary):
    prompt = f"""
    You are a medical AI expert trained to predict hospital readmission risk conservatively.
    Carefully analyze the following patient data and assess the likelihood of readmission within 30 days.

    **Classify as HIGH RISK (1) ONLY IF:**
    - The patient has multiple recent hospitalizations within the past 6 months.
    - There is a documented history of severe chronic conditions (e.g., heart failure, uncontrolled diabetes, kidney disease).
    - Medication records indicate frequent changes or high-risk prescriptions.
    - Prior admissions resulted in complications or worsening health trends.
    
    **Classify as LOW RISK (0) IF:**
    - The patient has stable health records without multiple recent admissions.
    - There are no significant chronic conditions or worsening trends.
    - Medication changes are minimal and indicate stability.
    
    Respond ONLY with '1' (High Risk) or '0' (Low Risk). Do not provide explanations.
    
    Patient Data:
    {patient_summary}
    """
    
    response = call_ollama(prompt)
    return int(response.strip()) if response.strip().isdigit() else 0  # Default to '0' if invalid response


In [130]:
df.loc[df.tail(500).index, "Predicted_Readmission"] = df.tail(500)["patient_summary"].apply(classify_readmission_with_ollama)



In [118]:
df.tail(50)[["patient_summary", "Predicted_Readmission"]]


,patient_summary,Predicted_Readmission
101712,"\n Patient ID: 92586510, a [60-70) Female p...",1.0
101713,"\n Patient ID: 160248038, a [40-50) Male pa...",0.0
101714,"\n Patient ID: 109990548, a [70-80) Female ...",0.0
101716,"\n Patient ID: 93806559, a [70-80) Male pat...",1.0
101717,"\n Patient ID: 182499863, a [50-60) Male pa...",1.0
101718,"\n Patient ID: 99489339, a [70-80) Female p...",0.0
101720,"\n Patient ID: 180189752, a [50-60) Male pa...",1.0
101721,"\n Patient ID: 106722468, a [70-80) Female ...",1.0
101722,"\n Patient ID: 40181832, a [70-80) Female p...",1.0
101723,"\n Patient ID: 98476776, a [60-70) Male pat...",0.0


In [131]:
df['readmitted_encoded'] = df['readmitted'].map({'>30': 0, 'NO': 0, '<30': 1})


In [132]:
y_true = df.tail(500)["readmitted_encoded"]
y_pred = df.tail(500)["Predicted_Readmission"]

In [121]:
y_true

101656    0
101657    0
101658    0
101660    0
101661    0
         ..
101761    0
101762    0
101763    0
101764    0
101765    0
Name: readmitted_encoded, Length: 100, dtype: int64

In [122]:
y_pred

101656    0.0
101657    1.0
101658    1.0
101660    0.0
101661    0.0
         ... 
101761    1.0
101762    1.0
101763    1.0
101764    1.0
101765    0.0
Name: Predicted_Readmission, Length: 100, dtype: float64

In [133]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_true, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")


Model Accuracy: 0.3220


In [134]:
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.91      0.28      0.43       455
           1       0.09      0.71      0.16        45

    accuracy                           0.32       500
   macro avg       0.50      0.50      0.30       500
weighted avg       0.83      0.32      0.41       500



In [135]:
from sklearn.metrics import recall_score, fbeta_score


In [136]:
recall = recall_score(y_true, y_pred)
f2_score = fbeta_score(y_true, y_pred, beta=2)  # F2-score (β = 2)

print(f"Recall: {recall:.4f}")
print(f"F2-Score: {f2_score:.4f}")


Recall: 0.7111
F2-Score: 0.2974
